In [ ]:
# -----------------------------------------------------------------------------
# 1. System & Path Setup
# -----------------------------------------------------------------------------
import os
import sys
import torch
import matplotlib.pyplot as plt

os.environ['HF_HUB_OFFLINE'] = '1'
os.environ['HF_HOME'] = os.path.expanduser('~/.cache/huggingface')
print("Hugging Face offline mode enabled.")

# -----------------------------------------------------------------------------
# 2. Project Imports 
# -----------------------------------------------------------------------------
from core_pipeline import *
from vis_utils import *
from datasets.dataset import WaymoOpenDataset

# -----------------------------------------------------------------------------
# 3. Environment Check
# -----------------------------------------------------------------------------
%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Environment ready. Device: {device}")

In [ ]:
# -----------------------------------------------------------------------------
# 实例化 Config (对应你提供的命令行参数)
# -----------------------------------------------------------------------------
scene_names = ["000"]
input_views = 1
squence_length = 20
start_idx=0
force_refresh = False
# output_path=f"outputs/scene{scene_name}_start{start_idx}_len{squence_length}_view{input_views}"
output_path="baseline"

config = DGGTConfig(
    # 必需路径参数
    image_dir="data/waymo/processed/validation",
    ckpt_path="/root/pretrained/model_latest_waymo.pt", 
    output_path=output_path,
    
    # 场景设置
    scene_names=scene_names,       # --scene_name 000
    input_views=input_views,             # --input_views 1
    sequence_length=squence_length,        # --sequence_length 20
    start_idx=start_idx,               # --start_idx 0
    
    # 功能开关 (根据你的命令行: -images -metrics -diffusion)
    images=True,
    metrics=True,
    
    # Refinement 开关 (命令行未开启，设为False，后续调试可改为True)
    enable_refinement=False,   
    depth=True
)

print(f"Config loaded for scenes: {config.scene_names}")
print(f"Output path: {config.output_path}")
print(f"Refinement enabled: {config.enable_refinement}")

In [ ]:
# 实例化 Dataset
# 假设 datasets.dataset 已经可以 import 并且包含了你提供的逻辑
# 如果是在 autodl 本地文件，确保路径正确
dataset = WaymoOpenDataset(
    image_dir=config.image_dir,
    scene_names=scene_names,
    sequence_length=config.sequence_length,
    start_idx=config.start_idx,
    mode=2,             # pure reconstruction
    views=config.input_views
)


In [ ]:
# 运行可视化
visualize_samples(dataset, [0, 4])

In [ ]:
# -----------------------------------------------------------------------------
# 4. Load Model, Run Inference & Visualize Results
# -----------------------------------------------------------------------------

# A simple logger for notebook use
class NotebookLogger:
    def info(self, msg):
        print(f"[INFO] {msg}")

# 1. Setup Environment & Model
logger = NotebookLogger()
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading checkpoint from: {config.ckpt_path}")
model = load_model(config, device, logger)


In [ ]:
print("Extracting depth data...")
dggt_depth, dggt_sky, da3_depth, da3_sky = extract_depth_alignment_data(config, model, dataset, device)

run_depth_alignment_debug(
    sample_indices=[0, 4],
    dggt_depth_seq=dggt_depth,
    dggt_sky_seq=dggt_sky,
    da3_depth_seq=da3_depth,
    da3_sky_seq=da3_sky,
    logger=logger,
    visualize_func=visualize_depth_alignment_6panel
)

In [ ]:
# 2. Run Inference
# Ensure config matches your current memory constraints (seq_len=2)
print(f"Running inference for scene: {config.scene_names} (Seq Len: {config.sequence_length})...")

# results is a list of dicts (one per scene)
config.diffusion =  True
config.penalize_alphas = False
config.use_nearest = True
config.metrics = True
config.enable_refinement = False
config.refine_pose = False
config.refine_gs = False
config.enable_da3 = True
results = run_inference_and_render(config, model, dataset, logger, device=device)
scene_data = results[0] # Get first scene

In [ ]:
# Run Vis
visualize_inference_results(scene_data,[0, 4])

In [ ]:
# --- Usage Example ---
# 假设 scene_data 已经由上面的推理代码生成
# save_indices = [0] # 只要第一帧
output_folder = "outputs/debug/da3_nearest_diffusion"

save_visualization_results(scene_data, [0, 4], output_folder)